In [2]:
import requests
import pandas as pd
import openpyxl
import numpy as np
import re
import yaml
import os

In [3]:
def get_response(url):
    response = requests.get(url)
    if response.status_code == 200:
        json_data = response.json()
        return extract_data(json_data)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
    

In [4]:
# Function to extract latitude, longitude, file URL, and variable name
def extract_data(study_data):
    for study in study_data['study']:
        for site in study['site']:
            # Extract latitude and longitude
            coordinates = site['geo']['geometry']['coordinates']
            latitude = coordinates[0]
            longitude = coordinates[1]
            
            print(f"Latitude: {latitude}, Longitude: {longitude}")

            file_url_list =list()
            file_variable = list()
            for paleo_data in site['paleoData']:
                for data_file in paleo_data['dataFile']:
                    # Extract file URL
                    file_url = data_file['fileUrl']
                    file_url_list.append(file_url)
                    print(f"File URL: {file_url}")
                    
                    # Extract variable names
                    variable_names = [var['cvShortName'] for var in data_file['variables']]
                    file_variable.append(variable_names)
                    print(f"Variables: {variable_names}")

    return [latitude, longitude, file_url_list, file_variable]


In [5]:
khider_url ="https://www.ncei.noaa.gov/access/paleo-search/study/search.json?xmlId=16017"
bhattacharya_url = "https://www.ncei.noaa.gov/access/paleo-search/study/search.json?xmlId=78037"
glaubek_url = "https://www.ncei.noaa.gov/access/paleo-search/study/search.json?xmlId=80598"
cody_url = "https://www.ncei.noaa.gov/access/paleo-search/study/search.json?xmlId=66868"

In [6]:
khider_response = get_response(khider_url)
bhattacharya_response = get_response(bhattacharya_url)
glaubek_response = get_response(glaubek_url)
cody_response = get_response(cody_url)

Latitude: 1.4033, Longitude: 119.078
File URL: https://www.ncei.noaa.gov/pub/data/paleo/contributions_by_author/khider2011/khider2011.txt
Variables: [None, None, None, None, None, None, None]
Latitude: 32.28, Longitude: -118.38
File URL: https://www.ncei.noaa.gov/pub/data/paleo/paleocean/pacific/bhattacharya2022/bhattacharya2022-odp1012.txt
Variables: ['dD_precip', 'dD_precip_err', 'age_ma', 'dD_wax', 'dD_wax_err', 'd13C_wax', 'd13C_wax_err']
Latitude: 23.03, Longitude: -109.03
File URL: https://www.ncei.noaa.gov/pub/data/paleo/paleocean/pacific/bhattacharya2022/bhattacharya2022-dsdp475.txt
Variables: ['age_ma', 'dD_wax', 'dD_wax_err', 'd13C_wax', 'd13C_wax_err', 'dD_precip', 'dD_precip_err']
Latitude: -.693833, Longitude: -85.33325
File URL: https://www.ncei.noaa.gov/pub/data/paleo/paleocean/pacific/glaubke2024/glaubke2024-09mc.txt
Variables: ['TimeSlice', 'Age_kyrs', 'SampSize_ug', 'Mg/Ca', 'Mn/Ca', 'Al/Ca', 'Fe/Ca', 'Mg/Ca-SWT']
Latitude: -.180533, Longitude: -85.86673
File URL: htt

In [7]:
responses = []
responses.append(khider_response)
responses.append(bhattacharya_response)
responses.append(glaubek_response)
responses.append(cody_response)

In [39]:
# for response in responses:
#     for files in response[2]:
#         data = pd.read_csv(files, delimiter="\t", comment="#")
#         print(data)
#         filename = os.path.splitext(os.path.basename(files))[0]
#         data.to_excel(f"{filename}.xlsx")


      depth_top  depth_bottom  age_ADbot  age_ADtop  d13CcarbVPDB  \
0             0             1       1843       1851         0.936   
1             0             1       1843       1851         0.895   
2             0             1       1843       1851         0.514   
3             0             1       1843       1851         0.900   
4             0             1       1843       1851         0.957   
...         ...           ...        ...        ...           ...   
1160         96            98        704        734         0.711   
1161         96            98        704        734         0.852   
1162         96            98        704        734         0.680   
1163         96            98        704        734         0.690   
1164         96            98        704        734         0.629   

      d18OcarbVPDB  wgt-ind  
0           -2.254       34  
1           -2.356       37  
2           -2.630       20  
3           -2.480       25  
4           -2.094   

In [8]:
yaml_file_path = '/Users/aayushbothraa/Desktop/Dhiren TUPF/isi-table-understanding/tmp/output/blue2019dust-corea.xlsx_0_Sheet1.yaml'
excel_file_path = '/Users/aayushbothraa/Desktop/Dhiren TUPF/isi-table-understanding/tmp/output/blue2019dust-corea.xlsx_colorised.xlsx'

# Load the YAML file to check the structure
with open(yaml_file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

# Load the Excel file
df = pd.read_excel(excel_file_path, sheet_name=0)  # Load the first sheet of the Excel file

# Extract the 'ObservationData' field from the YAML file
observation_data_str = yaml_data['variables']['data0']['ObservationData']

# Function to extract multiple row and column ranges from ObservationData
def extract_multiple_row_col_coordinates(block_string):
    # Find all occurrences of 'start..end' patterns
    all_ranges = re.findall(r'(\d+)\.\.(\d+)', block_string)
    # Separate row and column ranges for each block
    row_ranges = [(int(start), int(end)) for start, end in all_ranges[::2]]  # Every second range for rows
    col_ranges = [(int(start), int(end)) for start, end in all_ranges[1::2]]  # Every second range for columns
    return row_ranges, col_ranges

# Extract all row and column ranges from ObservationData
row_ranges, col_ranges = extract_multiple_row_col_coordinates(observation_data_str)

# Function to extract and print data for each block
def extract_data_for_all_blocks(df, row_ranges, col_ranges):
    for i, (row_range, col_range) in enumerate(zip(row_ranges, col_ranges)):
        start_row, end_row = row_range
        start_col, end_col = col_range
        # Extract block data
        block_data = df.iloc[start_row-1:end_row, start_col:end_col+1]
        print(f"Block {i+1} (rows {start_row} to {end_row}, columns {start_col} to {end_col}):\n")
        print(block_data)
        print("\n" + "="*50 + "\n")

# Extract and print data for all blocks
extract_data_for_all_blocks(df, row_ranges, col_ranges)

Block 1 (rows 1 to 1, columns 0 to 4):

   Unnamed: 0  age_calBP  dustfrac   sedrate     dens
0           0    2722.83   0.33702  0.049261  0.24186


